In [27]:
# 2022.06.05 try again
# Import packages
import dxpy
import dxdata
import pandas as pd
import pyspark

In [2]:
# Spark initialization (Done only once; do not rerun this cell unless you select Kernel Restart kernel).
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [3]:
# Automatically discover dispensed dataset ID and load the dataset
dispensed_dataset_id = dxpy.find_one_data_object(typename='Dataset', name='app*.dataset', folder='/', name_mode='glob')['id']
dataset = dxdata.load_dataset(id=dispensed_dataset_id)

In [4]:
participant = dataset['participant']

In [29]:
case = dxdata.load_cohort("220605_like_regular_fizzy_case")
cont = dxdata.load_cohort("220605_like_regular_fizzy_control")

In [34]:
field_ids = ['31', '21022', '22001', '22006', '22009', '22019', '22021', '22027',
             '41202', '41204', '20710', '20653',
             '20107', '2946', '1807',
             '20110', '3526', '1845']

# This function is used to grab all field names (e.g. "p<field_id>_iYYY_aZZZ") of a list of field IDs
def fields_for_id(field_id):
    from distutils.version import LooseVersion
    field_id = str(field_id)
    fields = participant.find_fields(name_regex=r'^p{}(_i\d+)?(_a\d+)?$'.format(field_id))
    return sorted(fields, key=lambda f: LooseVersion(f.name))

In [35]:
# Display field description in a good way
fields = [fields_for_id(f)[0] for f in field_ids] + [participant.find_field(name="p20160_i0")]
field_description = pd.DataFrame({
    'Field': [f.name for f in fields],
    'Title': [f.title for f in fields],
    'Coding': [f.coding.codes if f.coding is not None else '' for f in fields]
})
field_description

,Field,Title,Coding
0,p31,Sex,"{'0': 'Female', '1': 'Male'}"
1,p21022,Age at recruitment,
2,p22001,Genetic sex,"{'0': 'Female', '1': 'Male'}"
3,p22006,Genetic ethnic grouping,{'1': 'Caucasian'}
4,p22009_a1,Genetic principal components | Array 1,
5,p22019,Sex chromosome aneuploidy,{'1': 'Yes'}
6,p22021,Genetic kinship to other participants,{'-1': 'Participant excluded from kinship infe...
7,p22027,Outliers for heterozygosity or missing rate,{'1': 'Yes'}
8,p41202,Diagnoses - main ICD10,{'Chapter I': 'Chapter I Certain infectious an...
9,p41204,Diagnoses - secondary ICD10,{'Chapter I': 'Chapter I Certain infectious an...


In [15]:
# Retrieve fields into Pandas dataframe for both cohorts
case_df = participant.retrieve_fields(fields = fields, filter_sql = case.sql, engine=dxdata.connect()).toPandas())
cont_df = participant.retrieve_fields(
    fields = fields, filter_sql = cont.sql,
    engine=dxdata.connect(
        dialect="hive+pyspark",
        connect_args=
        {
            'config':{'spark.kryoserializer.buffer.max':'256m','spark.sql.autoBroadcastJoinThreshold':'-1'}
        }
    )).toPandas()

In [40]:
# Retrieve EID and variables for each cohort participant
# https://dnanexus.gitbook.io/uk-biobank-rap/working-on-the-research-analysis-platform/using-spark-to-analyze-tabular-data
case_df2 = participant.retrieve_fields(names=["eid", "p21022", "p31", "p22001", "p22006", "p22019", "p22021", "p22027", "p20160_i0", "p20710"], filter_sql=case.sql, engine=dxdata.connect()).toPandas()

cont_df2 = participant.retrieve_fields(
    names=["eid", "p21022", "p31", "p22001", "p22006", "p22019", "p22021", "p22027", "p20160_i0", "p20710"], 
    filter_sql = cont.sql,
    engine=dxdata.connect(
        dialect="hive+pyspark",
        connect_args=
        {
            'config':{'spark.kryoserializer.buffer.max':'256m','spark.sql.autoBroadcastJoinThreshold':'-1'}
        }
    )).toPandas()

In [42]:
case_df2

,eid,p21022,p31,p22001,p22006,p22019,p22021,p22027,p20160_i0,p20710
0,1001590,56,0,0.0,1.0,NaN,0.0,NaN,1.0,6
1,1009345,45,1,1.0,1.0,NaN,1.0,NaN,0.0,6
2,1009665,60,0,0.0,1.0,NaN,0.0,NaN,0.0,6
3,1009773,66,0,0.0,1.0,NaN,1.0,NaN,1.0,6
4,1011125,48,1,1.0,1.0,NaN,0.0,NaN,1.0,6
...,...,...,...,...,...,...,...,...,...,...
27170,6019032,52,0,0.0,1.0,NaN,0.0,NaN,0.0,9
27171,6020039,64,1,1.0,1.0,NaN,0.0,NaN,1.0,7
27172,6020422,55,0,0.0,1.0,NaN,1.0,NaN,0.0,7
27173,6021428,59,0,0.0,1.0,NaN,0.0,NaN,1.0,7


In [43]:
cont_df2

,eid,p21022,p31,p22001,p22006,p22019,p22021,p22027,p20160_i0,p20710
0,1000047,68,1,1.0,1.0,NaN,0.0,NaN,1.0,5
1,1000068,50,1,1.0,1.0,NaN,1.0,NaN,0.0,1
2,1000621,63,0,0.0,1.0,NaN,0.0,NaN,1.0,2
3,1000910,58,0,0.0,1.0,NaN,1.0,NaN,1.0,2
4,1001212,48,0,0.0,NaN,NaN,0.0,NaN,0.0,2
...,...,...,...,...,...,...,...,...,...,...
153970,6022645,64,1,1.0,1.0,NaN,0.0,NaN,0.0,2
153971,6022943,47,0,0.0,1.0,NaN,0.0,NaN,1.0,2
153972,6023243,62,0,0.0,1.0,NaN,1.0,NaN,1.0,2
153973,6023526,68,1,1.0,1.0,NaN,1.0,NaN,1.0,2


In [44]:
# Concatenate those dataframs into one
df = pd.concat([case_df2, cont_df2])
df.shape

(181150, 10)

In [46]:
cont_df2.eid

0         1000047
1         1000068
2         1000621
3         1000910
4         1001212
           ...   
153970    6022645
153971    6022943
153972    6023243
153973    6023526
153974    6024078
Name: eid, Length: 153975, dtype: object

In [47]:
print(type(df))
df

<class 'pandas.core.frame.DataFrame'>


,eid,p21022,p31,p22001,p22006,p22019,p22021,p22027,p20160_i0,p20710
0,1001590,56,0,0.0,1.0,NaN,0.0,NaN,1.0,6
1,1009345,45,1,1.0,1.0,NaN,1.0,NaN,0.0,6
2,1009665,60,0,0.0,1.0,NaN,0.0,NaN,0.0,6
3,1009773,66,0,0.0,1.0,NaN,1.0,NaN,1.0,6
4,1011125,48,1,1.0,1.0,NaN,0.0,NaN,1.0,6
...,...,...,...,...,...,...,...,...,...,...
153970,6022645,64,1,1.0,1.0,NaN,0.0,NaN,0.0,2
153971,6022943,47,0,0.0,1.0,NaN,0.0,NaN,1.0,2
153972,6023243,62,0,0.0,1.0,NaN,1.0,NaN,1.0,2
153973,6023526,68,1,1.0,1.0,NaN,1.0,NaN,1.0,2


In [48]:
# Create Phenotype variable:
df['regularfizzy_cc'] = 0
df.loc[df.eid.isin(case_df2.eid), 'regularfizzy_cc'] = 1

In [49]:
# I have 154k controls and 27k cases
df.regularfizzy_cc.value_counts()

0    153975
1     27175
Name: regularfizzy_cc, dtype: int64

In [51]:
# QC
df_qced = df[
    (df['p31'] == df['p22001']) &  # sex consistent
   # (df['p22006'] == 1 &           # white british ancestry
    (df['p22019'].isnull()) &      # no sex chromosome aneuploidy
    (df['p22021'] == 0)             # no kinship found           
]

In [52]:
# Check sample size after QC:
df_qced.regularfizzy_cc.value_counts()

0    106921
1     18656
Name: regularfizzy_cc, dtype: int64

In [55]:
# Rename columns for better readibility
import re
df_qced = df_qced.rename(columns=
                        {'eid': 'IID', 'p31': "sex", 'p21022': 'age',
                         'p20160_i0': 'ever_smoked',
                         'p22001': 'genetic_sex',
                         'p22027': 'outliers_hetero',
                         'p22006': 'ethnic_group',
                         'p22019': 'sex_chrom_aneuploidy',
                         'p22021': 'kinship_to_others',
                         'p20710': 'like_reg_fizzy_drinks'                            
                        })

# Add FID column -- required for regenie
df_qced['FID'] = df_qced['IID']

In [56]:
df_qced

,IID,age,sex,genetic_sex,ethnic_group,sex_chrom_aneuploidy,kinship_to_others,outliers_hetero,ever_smoked,like_reg_fizzy_drinks,regularfizzy_cc,FID
0,1001590,56,0,0.0,1.0,NaN,0.0,NaN,1.0,6,1,1001590
2,1009665,60,0,0.0,1.0,NaN,0.0,NaN,0.0,6,1,1009665
4,1011125,48,1,1.0,1.0,NaN,0.0,NaN,1.0,6,1,1011125
6,1013827,61,1,1.0,1.0,NaN,0.0,NaN,0.0,8,1,1013827
7,1014182,62,0,0.0,1.0,NaN,0.0,NaN,1.0,8,1,1014182
...,...,...,...,...,...,...,...,...,...,...,...,...
153968,6022340,40,1,1.0,1.0,NaN,0.0,NaN,1.0,1,0,6022340
153969,6022530,45,1,1.0,1.0,NaN,0.0,NaN,1.0,1,0,6022530
153970,6022645,64,1,1.0,1.0,NaN,0.0,NaN,0.0,2,0,6022645
153971,6022943,47,0,0.0,1.0,NaN,0.0,NaN,1.0,2,0,6022943


In [57]:
# Create a phenotype table from my QCed data
df_phenotype = df_qced[['FID', 'IID', 'regularfizzy_cc', 'sex', 'age', 'ever_smoked']]

In [58]:
df_phenotype

,FID,IID,regularfizzy_cc,sex,age,ever_smoked
0,1001590,1001590,1,0,56,1.0
2,1009665,1009665,1,0,60,0.0
4,1011125,1011125,1,1,48,1.0
6,1013827,1013827,1,1,61,0.0
7,1014182,1014182,1,0,62,1.0
...,...,...,...,...,...,...
153968,6022340,6022340,0,1,40,1.0
153969,6022530,6022530,0,1,45,1.0
153970,6022645,6022645,0,1,64,0.0
153971,6022943,6022943,0,0,47,1.0


In [60]:
# Try to get microarray data
# path_to_family_file = '/mnt/project/Bulk/Genotype Results/Imputation/UKB imputation from genotype' #replace this with actual path to the WES file
path_to_family_file = '/mnt/project/Bulk/Genotype Results/Genotype calls/ukb22418_c1_b0_v2.fam' #replace this with actual path to the WES file
plink_fam_df = pd.read_csv(path_to_family_file, delimiter='\s', dtype='object',
                           names = ['FID','IID','Father ID','Mother ID', 'sex', 'Pheno'], engine='python')

In [61]:
plink_fam_df

,FID,IID,Father ID,Mother ID,sex,Pheno
0,5343556,5343556,0,0,1,Batch_b001
1,2729828,2729828,0,0,2,Batch_b001
2,1342955,1342955,0,0,2,Batch_b001
3,4649126,4649126,0,0,2,Batch_b001
4,4274819,4274819,0,0,2,Batch_b001
...,...,...,...,...,...,...
488372,4762622,4762622,0,0,2,UKBiLEVEAX_b11
488373,3922066,3922066,0,0,2,UKBiLEVEAX_b11
488374,4478328,4478328,0,0,1,UKBiLEVEAX_b11
488375,3747345,3747345,0,0,2,UKBiLEVEAX_b11


In [62]:
# Intersect the phenotype data and the fam file to generate phenotype
# DataFrame for the Genetic calls participants
regularfizzy_microarray_df = df_phenotype.join(plink_fam_df.set_index('IID'), on='IID', rsuffix='_fam', how='inner')

# Drop unuseful columns from .fam file
regularfizzy_microarray_df.drop(
    columns=['FID_fam','Father ID','Mother ID', 'sex_fam', 'Pheno'], axis=1, inplace=True, errors='ignore'
)

In [63]:
# Finally, Write phenotype files to a TSV file!!!
regularfizzy_microarray_df.to_csv('regularfizzy_microarray_df.phe', sep='\t', na_rep='NA', index=False, quoting=3)
df_phenotype.to_csv('regularfizzy.phe', sep='\t', na_rep='NA', index=False, quoting=3)

In [77]:
%%bash
# Upload the geno-pheno intersect phenotype file back to the RAP project
dx upload regularfizzy_microarray_df.phe -p --path /regularfizzy_GWAS/ --brief
dx upload regularfizzy.phe -p --path /regularfizzy_GWAS/ --brief

file-GBKK0Z8J6jF2ZYjZPP6X4VZ0
file-GBKK0b0J6jF0Bg3ZPXF9xppv
